# Linear System Solvers

We cover some topics from Ch. 3 and 4 of Quatteroni (2000) about
- Direct solvers: Gauss elimination method
- Iterative solvers: Jacobi, Gauss-Seidel, Conjugate-Gradient, etc.

## Stability Analysis

We open this unit with some useful facts about the stability of a linear system. 

- Usually a subject in applied mathematics 
- Useful concepts are needed for and derived from the analysis


### The condition number of a matrix

For $A \in \mathbb{C}^{n\times n}$, the condition nmumber of $A$ is defined as
\begin{equation}
K(\mathbf{A}) = \Vert A \Vert \Vert A^{-1} \Vert,
\end{equation}
where $\Vert \cdot \Vert $ is an induced matrix norm.

A norm of a general $m\times n$ matrix is, in general, defined as
\begin{equation}
\sup_{\mathbf{v}\in \mathbb{C}^{n}\\\mathbf{v} \neq 0}\frac{\Vert \mathbf{Av}\Vert}{\Vert\mathbf{v}\Vert}
\end{equation}


#### Some frequently used norms

- $\Vert \cdot \Vert_{1}$: Column-sum norm
\begin{equation}
\Vert \mathbf{A} \Vert_{1} = \max_{j=1,\ldots,n}\sum_{i=1}^{m}\vert a_{ij} \vert,
\end{equation}

- $\Vert \cdot \Vert_{\infty}$: Row-sum norm
\begin{equation}
\Vert \mathbf{A} \Vert_{\infty} = \max_{i=1,\ldots,n}\sum_{j=1}^{m}\vert a_{ij} \vert,
\end{equation}

- $\Vert \cdot \Vert_{2}$
\begin{equation}
\begin{split}
\Vert \mathbf{A} \Vert_{2} &= \sqrt{\rho(\mathbf{A}^{H}\mathbf{A})} = \sqrt{\rho(\mathbf{A}\mathbf{A}^{H})} \\
&= \sigma_{1}(\mathbf{A}),
\end{split}
\end{equation}
where $\sigma_{1}(\mathbf{A})$ is the maximum singular value of $\mathbf{A}$
- When $\mathbf{A}$ is real and symmetric (i.e., $\mathbf{A}^{H}=\mathbf{A}$),
    $\Vert \mathbf{A} \Vert_{2} = \rho(\mathbf{A})$.
    - $\rho(\mathbf{A})$ is called **spectral radius of $\mathbf{A}$** and defined as
    \begin{equation}
    \rho(\mathbf{A}) = \max_{\lambda \in \sigma(\mathbf{A})} |\lambda|
    \end{equation}
    - $\sigma(\mathbf{A})$ above is the set of eigenvalues of $\mathbf{A}$ and called the **spectrum of $\mathbf{A}$**.
    

#### Examples of matrix condition numbers
- $K_{\infty}(\mathbf{A})=\Vert \mathbf{A} \Vert_{\infty}\Vert \mathbf{A}^{-1} \Vert_{\infty}$
- $K_{2}(\mathbf{A})=\Vert \mathbf{A} \Vert_{2}\Vert \mathbf{A}^{-1} \Vert_{2}$

Some other facts about include
- $1=\Vert \mathbf{A}\mathbf{A}^{-1} \Vert \le \Vert \mathbf{A} \Vert \Vert \mathbf{A}^{-1} \Vert = K(\mathbf{A}) \quad \therefore K(\mathbf{A}) \ge 1$.
- $K(\mathbf{A}^{-1}) = K(\mathbf{A})$.
- $^{\forall}\alpha \in \mathbb{C} \text{ with } \alpha \neq 0,\ K(\alpha\mathbf{A})=K(\mathbf{A})$.
- 
\begin{equation}
K_{2}(\mathbf{A}) = \Vert \mathbf{A} \Vert_{2} \Vert \mathbf{A}^{-1} \Vert_{2} = \frac{\sigma_{1}(\mathbf{A})}{\sigma_{n}(\mathbf{A})}
\end{equation}
- If $\mathbf{A}$ is symmetric, positive definite (i.e., $\lambda_{i} > 0$),
\begin{equation}
K_{2}(\mathbf{A}) = \frac{\lambda_{max}}{\lambda_{min}}=\rho(\mathbf{A})\rho(\mathbf{A}^{-1}).
\end{equation}
    - $K_{2}$ is called **spectral condition number**.


#### How far a matrix is from singularity

- Why do we care? 
    - Because $\mathbf{Ax}=\mathbf{0}$ does not necessarily mean $\mathbf{x}=\mathbf{0}$ if $\mathbf{A}$ is singular.
    
The relative distance of $\mathbf{A} \in \mathbb{C}^{n\times n}$ from the set of singular matrix with respect to the $p$-norm is defined as
\begin{equation}
\text{dist}_{p}(\mathbf{A}) = \min\left\{ \frac{\Vert \delta \mathbf{A} \Vert_{p}}{\Vert \mathbf{A} \Vert_{p}}: \mathbf{A} + \delta\mathbf{A}\text{ is singular}. \right\}
\end{equation}

It can be shown that $\text{dist}_{p}(\mathbf{A}) = 1/K_{p}(\mathbf{A})$.

#### Numpy Example

In [47]:
import numpy as np

a = np.array([[1, 0, -1], [0, 1, 0], [1, 0, 1]])
b = np.array([[5.0,0.0,0.0],[2.0,6.0,0.0],[2.0,1.0,8.0]]) # 3x3 matrix
print(a)
print(np.linalg.cond(a,1)) # max column sum
print(np.linalg.cond(a,2)) # max singular vaule
u, s, vh = np.linalg.svd(a) # singular value decomposition of a
print(s) # singular values of a
print(np.linalg.cond(a,np.inf)) # max row sum

[[ 1  0 -1]
 [ 0  1  0]
 [ 1  0  1]]
2.0
1.4142135623730951
[1.41421356 1.41421356 1.        ]
2.0


### Forward *a priori* Analysis

- If a numerical method is used for solving $\mathbf{Ax}=\mathbf{b}$, the exact solution cannot be acquired due to rounding errors.
- In other words, a numerical method yields an *exact* solution $\mathbf{x}+\delta\mathbf{x}$ of the perturbed system
\begin{equation}
(\mathbf{A}+\delta\mathbf{A})(\mathbf{x}+\delta\mathbf{x})=(\mathbf{b}+\delta\mathbf{b})
\end{equation}
- We would like to estimate $\delta\mathbf{x}$ in terms of $\delta\mathbf{A}$ and $\delta\mathbf{b}$.

- **Theorem**
    - Let $\mathbf{A} \in \mathbb{R}^{n\times n}$ be a non-singular matrix and $\delta \mathbf{A} \in \mathbb{R}^{n\times n}$ be such that $\mathbf{A}+\delta\mathbf{A}$ is still non-singular.
    - Then, if $\mathbf{x} \in \mathbb{R}^{n}$ is the solution of $\mathbf{Ax}=\mathbf{b}$ with $\mathbf{b} \in \mathbb{R}^{n}$ ($\mathbf{b}\neq \mathbf{0}$) and $\delta \mathbf{x} \in \mathbb{R}^{n}$ satisfied the perturbed system for $\delta \mathbf{b} \in \mathbb{R}^{n}$,
    \begin{equation}
      \frac{\Vert \delta \mathbf{x} \Vert}{\Vert \mathbf{x} \Vert} \le \frac{K(\mathbf{A})}{1-K(\mathbf{A})\Vert \delta \mathbf{A}\Vert / \Vert \mathbf{A} \Vert} \left( \frac{\Vert \delta \mathbf{b} \Vert}{\Vert \mathbf{b} \Vert} + \frac{\Vert \delta \mathbf{A} \Vert}{\Vert \mathbf{A} \Vert} \right).
    \end{equation}

- **Theorem**
    - If $\delta \mathbf{A}=0$, 
    \begin{equation}
      \frac{1}{K(\mathbf{A})} \frac{\Vert \delta \mathbf{b} \Vert}{\Vert \mathbf{b} \Vert} \le
      \frac{\Vert \delta \mathbf{x} \Vert}{\Vert \mathbf{x} \Vert} \le 
      K(\mathbf{A}) \frac{\Vert \delta \mathbf{b} \Vert}{\Vert \mathbf{b} \Vert}
    \end{equation}

### *a posteriori* Analysis
- Analysis done **after** an approximation solution is acquired.
- The aim is to relate unknown error $\mathbf{e} = \mathbf{y}-\mathbf{x}$ to quantities that can be computed using $\mathbf{y}$ and $\mathbf{C}$ where $\mathbf{C}$ is the approximate inverse of $\mathbf{A}$ and $\mathbf{y}$ is a known approximate solution from $\mathbf{y}=\mathbf{C}\mathbf{b}$.
    - cf. $\mathbf{x}=\mathbf{A}^{-1}\mathbf{b}$.
- Residual vector ($\mathbf{r}$)
\begin{equation}
\mathbf{r} = \mathbf{b} - \mathbf{Ay},
\end{equation}
    - In general, $\mathbf{r}$ is non-zero.
- Residual and error are related as follows:
\begin{equation}
\mathbf{e} = \mathbf{y}-\mathbf{x} = \mathbf{A}^{-1}(\mathbf{Ay}-\mathbf{b}) = -\mathbf{A}^{-1}\mathbf{r}
\end{equation}
- $\mathbf{R}=\mathbf{AC}-\mathbf{I}$. If $\Vert \mathbf{R} \Vert < 1$,
\begin{equation}
\Vert e \Vert \le \frac{\Vert \mathbf{r} \Vert\,\Vert \mathbf{C} \Vert}{1-\Vert \mathbf{R} \Vert}.
\end{equation}

## Direct method

- Solution is acquired in a **finite** number of steps. Usually for small problems.
- Iterative methods: Solution acquired in **infinite** steps.
    - Usually requires fewer steps than a direct method
    - Often used for large problems

### Forward substitution

\begin{equation}
\mathbf{Lx} = \mathbf{b}
\end{equation}

\begin{equation}
\begin{bmatrix}
l_{11} & 0      & 0      & \cdots \\
l_{21} & l_{22} & 0      & \cdots \\
l_{31} & l_{32} & l_{33} & \cdots \\
       &   \vdots &      &
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
x_{3} \\
\vdots
\end{bmatrix}
=
\begin{bmatrix}
b_{1} \\
b_{2} \\
b_{3} \\
\vdots
\end{bmatrix}
\end{equation}

\begin{align}
x_{1} &= b_{1}/l_{11} \\
x_{2} &= (b_{2}-l_{21}x_{1})/l_{22} \\
x_{3} &= (b_{3}-l_{31}x_{1}-l_{32}x_{2})/l_{33} \\
 &\vdots \\
x_{n} &= \left( b_{n}-\sum_{k=1}^{n-1}l_{nk}x_{k} \right)/l_{nn} \\ 
\end{align}

- Numerical cost is $n^{2}$ flops.

### Backward substitution

\begin{equation}
\mathbf{Ux} = \mathbf{b}
\end{equation}

\begin{equation}
\begin{bmatrix}
u_{11} & u_{12} & u_{13} & \cdots & u_{1n} \\
0      & u_{22} & u_{23} & \cdots & u_{2n} \\
0      & 0      & u_{33} & \cdots & u_{3n} \\
       &        & \vdots &        &        \\
0      & 0      & 0      & \cdots & u_{nn}
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
x_{3} \\
\vdots \\
x_{n}
\end{bmatrix}
=
\begin{bmatrix}
b_{1} \\
b_{2} \\
b_{3} \\
\vdots \\
b_{n}
\end{bmatrix}
\end{equation}

\begin{align}
x_{n} &= b_{n}/u_{nn} \\
x_{n-1} &= (b_{n-1}-u_{n-1\,n}x_{n})/u_{n-1\,n-1} \\
 &\vdots \\
x_{i} &= \left( b_{i}-\sum_{k=i+1}^{n}u_{ik}x_{k} \right)/u_{ii}
\end{align}

- Numerical cost is $n^{2}$ flops.

\begin{equation}
x_{n} = \left( b_{n}-\sum_{k=1}^{n-1}l_{nk}x_{k} \right)/l_{nn}
\end{equation}

#### Program 1 - forward row : Forward substitution: row-oriented version
```Matlab
function [x]=forward_row(L,b)
[n]=mat_square(L); x(1) = b(1)/L(1,1);
for i = 2:n, x (i) = (b(i)-L(i,1:i-1)*(x(1:i-1))’)/L(i,i); end
x=x’;
```

#### Program 2 - forward col : Forward substitution: column-oriented version
```Matlab
function [b]=forward_col(L,b)
[n]=mat_square(L);
for j=1:n-1,
b(j)= b(j)/L(j,j); b(j+1:n)=b(j+1:n)-b(j)*L(j+1:n,j);
end; b(n) = b(n)/L(n,n);
```

\begin{equation}
x_{i} = \left( b_{i}-\sum_{k=i+1}^{n}u_{ik}x_{k} \right)/u_{ii}
\end{equation}

#### Program 3 - backward col : Backward substitution: column-oriented version
```Matlab
function [b]=backward_col(U,b)
[n]=mat_square(U);
for j = n:-1:2,
b(j)=b(j)/U(j,j); b(1:j-1)=b(1:j-1)-b(j)*U(1:j-1,j);
end; b(1) = b(1)/U(1,1);
```

In [49]:
import numpy as np

L = np.array([[5.0,0.0,0.0],[2.0,6.0,0.0],[2.0,1.0,8.0]]) # 3x3 matrix
x = np.array([[0.1, 0.2, 0.3]]).T # Make x a column vector
# Important!! 
# Matrix-vector multiplication as we know is 
#  np.dot(L,x) or L.dot(x)
b = L.dot(x)  # L*x
print('L=',L)
print('x=',x)
print('b=',b)

L= [[5. 0. 0.]
 [2. 6. 0.]
 [2. 1. 8.]]
x= [[0.1]
 [0.2]
 [0.3]]
b= [[0.5]
 [1.4]
 [2.8]]


In [50]:
def forward_row(L,b):
    # Check if L is a square matrix
    n,m = L.shape
    if n!=m:
        print('L is not square')
        return
    x = np.zeros((n,1)) # column vector
    x[0,0] = b[0,0]/L[0,0]
    for i in range(1,n): # 1 to n-1
        x[i,0] = (b[i,0]-L[i,0:i].dot(x[0:i,0]))/L[i,i]
    return x
print(forward_row(L,b))

[[0.1]
 [0.2]
 [0.3]]


### Gauss elimination

- Reduces the system $\mathbf{Ax}=\mathbf{b}$ to an equivalent system of the form $\mathbf{Ux}=\hat{\mathbf{b}}$.
- Then, the reduced system can be solved by the backward substitution.
- Complexity of GEM: $\frac{2}{3}n^{3} + 2n^{2} \sim \frac{2}{3}n^{3}$.

- We review the algorithm using this example:
\begin{equation}
\mathbf{A}^{(1)} \mathbf{x} = \mathbf{b}^{(1)} \quad
\begin{matrix}
x_{1}  &+& \frac{1}{2}x_{2} &+& \frac{1}{3}x_{3} &=& \frac{11}{6} \\
\frac{1}{2}x_{1} &+& \frac{1}{3}x_{2} &+& \frac{1}{4}x_{3} &=& \frac{13}{12} \\
\frac{1}{3}x_{1} &+& \frac{1}{4}x_{2} &+& \frac{1}{5}x_{3} &=& \frac{47}{60}
\end{matrix}
\end{equation}

    - Go ahead and start solving this problem!
    - Note that we want to make an upper triangular matrix.

\begin{equation}
\mathbf{A}^{(2)} \mathbf{x} = \mathbf{b}^{(2)} \quad
\begin{matrix}
x_{1}  &+& \frac{1}{2}x_{2} &+& \frac{1}{3}x_{3} &=& \frac{11}{6} \\
0  &+& \frac{1}{12}x_{2} &+& \frac{1}{12}x_{3} &=& \frac{1}{6} \\
0  &+& \frac{1}{12}x_{2} &+& \frac{4}{45}x_{3} &=& \frac{31}{180}
\end{matrix}
\end{equation}

\begin{equation}
\mathbf{A}^{(3)} \mathbf{x} = \mathbf{b}^{(3)} \quad
\begin{matrix}
x_{1}  &+& \frac{1}{2}x_{2} &+& \frac{1}{3}x_{3} &=& \frac{11}{6} \\
0  &+& \frac{1}{12}x_{2} &+& \frac{1}{12}x_{3} &=& \frac{1}{6} \\
0  &+& 0 &+& \frac{1}{180}x_{3} &=& \frac{1}{180}
\end{matrix}
\end{equation}

So, the solution is $x_{1}=x_{2}=x_{3}=1$.

### Equivalence of GEM to LU factorization
In the above example, note that $\mathbf{A}^{(2)} = \mathbf{M}_{1}\mathbf{A}^{(1)}$, where
\begin{equation}
\mathbf{M}_{1} = 
\begin{pmatrix}
1 & 0 & 0 \\
-\frac{1}{2} & 1 & 0 \\
-\frac{1}{3} & 0 & 1 \\
\end{pmatrix}.
\end{equation}

Also, $\mathbf{A}^{(3)} = \mathbf{M}_{2}\mathbf{A}^{(2)} = \mathbf{M}_{2}\mathbf{M}_{1}\mathbf{A}^{(1)} = \mathbf{U}$, where
\begin{equation}
\mathbf{M}_{2} = 
\begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & -1 & 1 \\
\end{pmatrix}.
\end{equation}

Therefore, we get this equivalence:
\begin{equation}
\mathbf{A} = \left(\mathbf{M}_{2}\mathbf{M}_{1}\right)^{-1}\mathbf{U} = \mathbf{LU}
\end{equation}
because $\mathbf{M}_{1}$ and $\mathbf{M}_{2}$ are lower triangular matrices.

- So, when $\mathbf{Ax}=\mathbf{b}$ is given, applying GEM is equivalent to perform LU factorization of $\mathbf{A}$.
\begin{equation}
\mathbf{LUx}=\mathbf{b}
\end{equation}
    - Solve $\mathbf{Ly}=\mathbf{b}$ first adn then $\mathbf{Ux}=\mathbf{y}$ for $\mathbf{x}$.
- Special considerations are needed for **sparse** matrices
    - $n\times n$ sparse matrix has about $n$ non-zero elements: e.g., tridiagonal or other banded matrices.
    - For instance, how to perform LU factorization in the *compact storage* form.

## Iterative Methods

- The general idea is to construct a sequence of vectors, $\mathbf{x}^{(k)}$ that has the property
\begin{equation}
\mathbf{x} = \lim_{k\rightarrow \infty} \mathbf{x}^{(k)},
\end{equation}
where $\mathbf{x}$ is the exact solution.
- $\mathbf{x}^{(0)}$ given, 
\begin{equation}
\mathbf{x}^{(k+1)} = \mathbf{B}\mathbf{x}^{(k)} + \mathbf{f}(\mathbf{b}), \quad k \ge 0,
\end{equation}
where $\mathbf{B}$ is a $n\times n$ square matrix called **iteration matrix**.


- An iterative method of the above form is said to be **consistent** with $\mathbf{Ax}=\mathbf{b}$ if $\mathbf{f}$ and $\mathbf{B}$ are such that $\mathbf{x} = \mathbf{Bx} + \mathbf{f}$.

- The importance of iteration matrix is in that its **spectral radius** ($\rho(\mathbf{B})$) determines both **whether the method will converge** and **how fast it will converge** (i.e., convergence rate).

#### Theorem 4.1 
In the above iteration scheme, the sequence of vectors $\left\{ \mathbf{x}^{(k)} \right\}$ converges to the solution of $\mathbf{Ax}=\mathbf{b}$ for any choice of $\mathbf{x}^{(0)}$ iff $\rho(\mathbf{B}) < 1$.

- **Asymptotic convergence rate and factor**
\begin{equation}
R(\mathbf{B}) = \lim_{k\rightarrow \infty} R_{k}(\mathbf{B}) = -\log \rho(\mathbf{B}).
\end{equation}
and $R(\mathbf{B})$ is called *asymptotic convergence factor*.

    - Note that the convergence rate is *positive* only when $\rho(\mathbf{B}) < 1$.
    - In other words, if $\rho(\mathbf{B})\ge 1$, the iteration method doesn't converge.

### Linear iterative methods
- *Linear* because these are based on **additive splitting** of $\mathbf{A}$.
- $\mathbf{A} = \mathbf{P} - \mathbf{N}$, where $\mathbf{P}$ is called **preconditioner**.
- According to this decomposition, we get the following result:

\begin{equation}
\mathbf{Ax}=\mathbf{b} \rightarrow (\mathbf{P}-\mathbf{N})x = \mathbf{b} \rightarrow \mathbf{Px} = \mathbf{Nx} + \mathbf{b}.
\end{equation}

- From this, we get an iteration scheme:
    
\begin{equation}
\begin{split}
\mathbf{P}\mathbf{x}^{(k+1)} &= \mathbf{N}\mathbf{x}^{(k)} + \mathbf{b}.\\
          \mathbf{x}^{(k+1)} &= \mathbf{P}^{-1}\mathbf{N}\mathbf{x}^{(k)} + \mathbf{P}^{-1}\mathbf{b} \\
                             &= \mathbf{B}\mathbf{x}^{(k)} + \mathbf{f}
\end{split}
\end{equation}

- Another possible scheme is 
    
\begin{equation}
\begin{split}
\mathbf{P}\mathbf{x}^{(k+1)} &= (\mathbf{P}-\mathbf{A})\mathbf{x}^{(k)} + \mathbf{b}. \\
                             &= \mathbf{P}\mathbf{x}^{(k)} + (\mathbf{b}-\mathbf{A}\mathbf{x}^{(k)}). \\
\therefore \mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \mathbf{P}^{-1}\mathbf{r}^{(k)},
\end{split}
\end{equation}
where $\mathbf{r}^{(k)}$ is the $k$-th **residual vector** (i.e., $\mathbf{b}-\mathbf{A}\mathbf{x}^{(k)}$).


- In the above scheme, a linear system, with coefficient matrix $\mathbf{P}$, must be solved to update the solution at step $k+1$. 
- Thus $\mathbf{P}$, besides being **non-singular** (i.e., invertible), ought to be **easily (i.e., cheaply) invertible** in order to keep the overall computational cost low.
- For instace, if $\mathbf{P}$ were equal to $\mathbf{A}$ and $\mathbf{N}=\mathbf{0}$, the above iteration method would converge in one iteration, but at the same cost of a direct method.

### Jacobi method
- One realization of the above decomposition is $\mathbf{A} = \mathbf{D} - (\mathbf{D}-\mathbf{A}) = \mathbf{D} - (\mathbf{E}+\mathbf{F})$
    - where $\mathbf{D}$, $\mathbf{E}$, and $\mathbf{F}$ are the *diagonal* elements, *negative lower triangular* part and *negative upper triangular* part of $\mathbf{A}$, respectively.
    - The iteration scheme acquired from this decomposition is
    
    \begin{equation}
    \mathbf{Dx} = (\mathbf{E}+\mathbf{F})\mathbf{x} + \mathbf{b}
    \end{equation}
    \begin{equation}
    \mathbf{D}\mathbf{x}^{(k+1)} = (\mathbf{E}+\mathbf{F})\mathbf{x}^{(k)} + \mathbf{b}
    \end{equation}
    \begin{align}
    \mathbf{x}^{(k+1)} &= \mathbf{D}^{-1}\left[ (\mathbf{E}+\mathbf{F})\mathbf{x}^{(k)} + \mathbf{b} \right] \\
                       &= \mathbf{D}^{-1}(\mathbf{E}+\mathbf{F})\mathbf{x}^{(k)} + \mathbf{D}^{-1}\mathbf{b} \\
                       &= \mathbf{B}_{J}\mathbf{x}^{(k)} + \mathbf{b}_{J}.
    \end{align}


-    
    - This iteration scheme can be written in the following element form:
    
    \begin{equation}
    x_{i}^{(k+1)} = \frac{1}{a_{ii}}\left[ b_{i} - \sum_{j=1\\j\neq i}^{n} a_{ij}x_{j}^{(k)}  \right]\quad i=1,\cdots,n.
    \end{equation}
    - Note that for this method to hold always, $\mathbf{A}$'s **diagonal elements should be non-zero**.   

### Gauss-Seidel method
- Note that in the Jacobi method, for $i>1$, $x_{i}^{(k+1)}$ are known.
- So, it is possible to use them to update the solution:
\begin{equation}
    x_{i}^{(k+1)} = \frac{1}{a_{ii}}\left[ b_{i} - \sum_{j=1}^{i-1} a_{ij}x_{j}^{(k+1)} - \sum_{j=i+1}^{n} a_{ij}x_{j}^{(k)} \right] \quad i=1,\cdots,n
\end{equation}
- The decomposition corresponding to the GS is
\begin{equation}
\mathbf{P} = \mathbf{D} - \mathbf{E} \quad \text{and} \quad \mathbf{N} = \mathbf{F}.
\end{equation}
- The associated iteration matrix is
\begin{equation}
\mathbf{B}_{GS} = \left( \mathbf{D} - \mathbf{E} \right)^{-1} \mathbf{F}.
\end{equation}


### Over-relaxation method
- $\omega$: Relaxation parameter
    - $\omega > 1$: Over-relaxation
    - $\omega < 1$: Under-relaxation
    
#### Jacobi over-relaxation (JOR) method
\begin{equation}
    x_{i}^{(k+1)} = \frac{\omega}{a_{ii}}\left[ b_{i} - \sum_{j=1\\j\neq i}^{n} a_{ij}x_{j}^{(k)}  \right] + (1-\omega)x_{i}^{(k)}.
\end{equation}

#### Successive over-relaxation (SOR) method
\begin{equation}
    x_{i}^{(k+1)} = \frac{\omega}{a_{ii}}\left[ b_{i} - \sum_{j=1}^{i-1} a_{ij}x_{j}^{(k+1)} - \sum_{j=i+1}^{n} a_{ij}x_{j}^{(k)} \right] + (1-\omega)x_{i}^{(k)}.
\end{equation}

### Convergence properties of Jacobi, GS, JOR and SOR

- Refer to Sec. 4.2.2 and 4.2.3 of Quarteroni (2000).
    - As usual, *nice* (e.g., symmetric and positive definite) matrices are guaranteed to converge.
- We apply what we've just learned to the following examples of matrices (Example 4.2 of Quarteroni, 2000)


In [143]:
import numpy as np
import numpy.linalg as LA

A1o = np.array([[ 3.0, 0.0, 4.0],[ 7.0, 4.0, 2.0],[-1.0, 1.0, 2.0]])
A1  = np.array([[ 4.0, 2.0, 3.0],[ 7.0, 4.0, 2.0],[-1.0, 1.0, 2.0]])
A2  = np.array([[-3.0, 3.0,-6.0],[-4.0, 7.0,-8.0],[ 5.0, 7.0,-9.0]])
A3  = np.array([[ 4.0, 1.0, 1.0],[ 2.0,-9.0, 0.0],[ 0.0,-8.0,-6.0]])
A4  = np.array([[ 7.0, 6.0, 9.0],[ 4.0, 5.0,-4.0],[-7.0,-3.0, 8.0]])
print(A1o,'\n',A1,'\n', A2,'\n', A3,'\n', A4)

[[ 3.  0.  4.]
 [ 7.  4.  2.]
 [-1.  1.  2.]] 
 [[ 4.  2.  3.]
 [ 7.  4.  2.]
 [-1.  1.  2.]] 
 [[-3.  3. -6.]
 [-4.  7. -8.]
 [ 5.  7. -9.]] 
 [[ 4.  1.  1.]
 [ 2. -9.  0.]
 [ 0. -8. -6.]] 
 [[ 7.  6.  9.]
 [ 4.  5. -4.]
 [-7. -3.  8.]]


In [145]:
def get_convergence_factors(A):
    n, m = A.shape
    if n != m:
        print("A is not a square matrix! Aborting.")
        return
    D = np.diag(np.diag(A))
    E = -(np.tril(A) - D)
    F = -(np.triu(A) - D)
    #print(A,'\n',D,'\n',E,'\n',F,'\n',D-(E+F))
    #print(LA.inv(D),'\n',E+F)
    BJ = LA.inv(D).dot((E+F))
    #print(BJ,LA.eigvals(BJ),np.eye(3)-LA.inv(D).dot(A))
    #print(LA.inv(D-E),'\n',F)
    BGS = LA.inv(D-E).dot(F)
    #print(BGS,LA.eigvals(BGS))
    #print(np.absolute(LA.eigvals(BJ)), np.absolute(LA.eigvals(BGS)))
    return [np.max(np.absolute(LA.eigvals(BJ))), np.max(np.absolute(LA.eigvals(BGS)))]

In [147]:
print('rho(BG)','rho(BGS)')
print(get_convergence_factors(A1o))
print(get_convergence_factors(A1))
print(get_convergence_factors(A2))
print(get_convergence_factors(A3))
print(get_convergence_factors(A4))

rho(BG) rho(BGS)
[1.1251473563526098, 1.5833333333333335]
[1.1086807884578176, 0.40352225657854146]
[0.8133091054692768, 1.1111111111111105]
[0.4438188250156999, 0.018518518518518517]
[0.641132809955697, 0.7745966692414834]


## Gradient Method

- We can generalize the idea of relaxation. Consider the following slightly modified form of the iteration scheme:
\begin{equation}
\begin{split}
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \mathbf{P}^{-1}\mathbf{r}^{(k)} \\
&\Downarrow \\
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \alpha\mathbf{P}^{-1}\mathbf{r}^{(k)},
\end{split}
\end{equation}
where $\alpha$ is called **relaxation** or **acceleration parameter**.
- Also the iteration scheme is said to be **stationary** if $\alpha$ is independent of index $k$.
    - cf. **Non-stationary method** if $\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_{k}\mathbf{P}^{-1}\mathbf{r}^{(k)}$.

- **Gradient method** is one of the non-stationary method
- Note $\mathbf{Ax}=\mathbf{b}$'s solution $\mathbf{x}$ is also the minimizer of
\begin{equation}
\Phi(\mathbf{y}) = \frac{1}{2}\mathbf{y}^{T}\mathbf{Ay} - \mathbf{y}^{T}\mathbf{b},
\end{equation}
if $\mathbf{A}$ is symmetric and positive definite.
    - because $\nabla\Phi(\mathbf{y}) = \frac{1}{2}(\mathbf{Ay}+\mathbf{A}^{T}\mathbf{y}) - \mathbf{b} = \mathbf{Ay}-\mathbf{b}.$
    - In other words, a *minimizer* that makes $\nabla\Phi=\mathbf{0}$ is the solution to $\mathbf{Ay}=\mathbf{b}$.
    - Conversely, if $\mathbf{x}$ is a solution, then
    \begin{equation}
    \begin{split}
      &\Phi(\mathbf{y}) = \Phi(\mathbf{x} + (\mathbf{y}-\mathbf{x})) = \Phi(\mathbf{x}) + \frac{1}{2}(\mathbf{y}-\mathbf{x})^{T}\mathbf{A}(\mathbf{y}-\mathbf{x}) \\
      &\therefore \Phi(\mathbf{y}) > \Phi(\mathbf{x})\text{ for } ^{\forall}\mathbf{y} \neq \mathbf{x}. \\
      &\therefore \mathbf{x} \text{ is a minimizer}.
    \end{split}
    \end{equation}
    
    

- How to find $\mathbf{x}$ starting from an initial guess, $\mathbf{x}^{(0)}$?
- In other words, the goal is to find $\mathbf{x}$ such that
\begin{equation}
\mathbf{x} = \mathbf{x}^{(0)} + \alpha \mathbf{d}.
\end{equation}
- Of course, $\alpha$ and $\mathbf{d}$ are not known *a priori*.
- So, we take steps:
\begin{equation}
\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_{k} \mathbf{d}^{(k)}\quad k=0,1,2,\cdots.
\end{equation}
- One natural idea is to take the descent direction of maximum slope: i.e., $\mathbf{d} = -\nabla \Phi(\mathbf{x}^{(k)})$.
    - For this reason, the method is called **gradient method** or **steepest descent method**.

- We saw $\nabla \Phi(\mathbf{x}^{(k)}) = \mathbf{A}\mathbf{x}^{(k)} - \mathbf{b} = -\mathbf{r}^{(k)}$.
    - $\therefore$ in the GM, $\mathbf{x}^{(k)}$ **moves along the direction** $\mathbf{r}^{(k)}$ to get to $\mathbf{x}^{(k+1)}$.
    
- Since we've found the direction, we now need to know how far to move along that direction, which is quantified by $\alpha_{k}$.


- To find $\alpha_{k}$, note that
\begin{equation}
\Phi(\mathbf{x}^{(k+1)}) = \frac{1}{2} \left( \mathbf{x}^{(k)}+\alpha \mathbf{r}^{(k)} \right)^{T}\mathbf{A} \left( \mathbf{x}^{(k)}+\alpha \mathbf{r}^{(k)} \right) - \left(\mathbf{x}^{(k)}+\alpha \mathbf{r}^{(k)} \right)^{T}\mathbf{b}.
\end{equation}

    - Take the partical derivative of the above expression of $\Phi$ with respect to $\alpha$.
    - $\alpha$ that make that derivative equal to zero will provide the local minimum of $\Phi$ along the direction $\mathbf{d}$.
    \begin{equation}
    \begin{split}
    \frac{\partial\Phi}{\partial\alpha_{k}} &= \frac{1}{2} \mathbf{r}^{(k)T}\mathbf{A} \left( \mathbf{x}^{(k)}+\alpha_{k} \mathbf{r}^{(k)} \right) + \frac{1}{2} \left( \mathbf{x}^{(k)}+\alpha_{k} \mathbf{r}^{(k)} \right)^{T}\mathbf{A} \mathbf{r}^{(k)} - \mathbf{r}^{(k)T}\mathbf{b} \\
    &= \mathbf{r}^{(k)T} \mathbf{A} \left( \mathbf{x}^{(k)}+\alpha_{k} \mathbf{r}^{(k)} \right)  - \mathbf{r}^{(k)T}\mathbf{b} \\
    &= \mathbf{r}^{(k)T} \left( \mathbf{A}\mathbf{x}^{(k)}-\mathbf{b} \right) + \alpha_{k} \left( \mathbf{r}^{(k)T}\mathbf{A}\mathbf{r}^{(k)} \right) \\
    &= -\mathbf{r}^{(k)T}\mathbf{r}^{(k)} + \alpha_{k} \left( \mathbf{r}^{(k)T}\mathbf{A}\mathbf{r}^{(k)} \right) \\
    &= 0.
    \end{split}
    \end{equation}
    \begin{equation}
    \therefore \alpha_{k} = \frac{\mathbf{r}^{(k)T}\mathbf{r}^{(k)}}{ \mathbf{r}^{(k)T}\mathbf{A}\mathbf{r}^{(k)} }
    \end{equation}

- In summary, in GM, given $\mathbf{x}^{(0)}$, for $k=0,1,\cdots$, until convergence, compute
\begin{align}
\mathbf{r}^{(k)} &= \mathbf{b} - \mathbf{A}\mathbf{x}^{(k)} \\
\alpha_{k} &= \frac{\mathbf{r}^{(k)T}\mathbf{r}^{(k)}}{ \mathbf{r}^{(k)T}\mathbf{A}\mathbf{r}^{(k)} } \\
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \alpha_{k} \mathbf{r}^{(k)}.
\end{align}

- Let's try to have some geometric intuition of the GM.
    - When $\Phi$ is such that the contours are concentric circles
        - 1 step is sufficient.
    - When the countours are highly elongated ellipses,
        - Convergence can be very slow.
    
    
- GM consists of 2 operations:
    1. Finding search direction (steepest descent)
    2. Picking up a point of local minimum along that direction.
    
    - However, the two operations are **independent**. 
    - This realization is the starting point of the **conjugate gradient method**.

## Conjugate Gradient Method

- CG and GM has the same starting point
    - The solution to $\mathbf{Ax}=\mathbf{b}$ is also the minimizer of $\Phi(\mathbf{y}) = \frac{1}{2}\mathbf{y}^{T}\mathbf{Ay} - \mathbf{y}^{T}\mathbf{b}$ when $\mathbf{A}$ is summetric and positive definite.
- The key difference between CG and GM is
    - GM follows **the steepest descent directions**
    - CG follows **mutually A-orthogonal directions**
    - Two vectors $x$ and $y$ are **conjugate** or **A-orthogonal** if $\mathbf{x}^{T}\mathbf{Ay}=\mathbf{0}$ for a symmetric non-singular matrix.

- Our task is to understand why it is better to follow mutually A-orthogonal directions. 


Let's recall that, for a given *search* direction $\mathbf{p}^{(k)}$, finding how far to move along that direction is straightforward:
If $\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_{k}\mathbf{p}^{(k)}$,

\begin{equation}
\Phi\left(\mathbf{x}^{(k+1)}\right) = \frac{1}{2}\left(\mathbf{x}^{(k)} + \alpha_{k}\mathbf{p}^{(k)} \right)^{T}\mathbf{A}\left(\mathbf{x}^{(k)} + \alpha_{k}\mathbf{p}^{(k)} \right) - \left(\mathbf{x}^{(k)} + \alpha_{k}\mathbf{p}^{(k)} \right)^{T}\mathbf{b}
\end{equation}

\begin{equation}
\frac{\partial\Phi}{\partial \alpha_{k}} = \frac{1}{2}\mathbf{p}^{(k)T}\mathbf{A}\left(\mathbf{x}^{(k)}+ \alpha_{k}\mathbf{p}^{(k)} \right) 
+ \frac{1}{2}\left(\mathbf{x}^{(k)} + \alpha_{k}\mathbf{p}^{(k)} \right)^{T}\mathbf{A}\mathbf{p}^{(k)} - \mathbf{p}^{(k)T}\mathbf{b} = 0
\end{equation}

\begin{equation}
\left( \mathbf{p}^{(k)T}\mathbf{A}\mathbf{p}^{(k)} \right)\alpha_{k} = \mathbf{p}^{(k)T}\left( \mathbf{b}-\mathbf{A}\mathbf{x}^{(k)} \right) = \mathbf{p}^{(k)T}\mathbf{r}^{(k)}
\end{equation}

\begin{equation}
\therefore \alpha_{k} = \frac{\mathbf{p}^{(k)T}\mathbf{r}^{(k)}}{\mathbf{p}^{(k)T}\mathbf{A}\mathbf{p}^{(k)}}
\end{equation}

- The remaining question is how to find a suitable $\mathbf{p}^{(k)}$.
    - When $\mathbf{p}^{(k)} = \mathbf{r}^{(k)}$, we get GM.
- To be able to do so, we need to first define what *suitable* means.
- **Definition**: $\mathbf{x}^{(k)}$ is said to be **optimal** with respect to a certain direction $\mathbf{p}\neq\mathbf{0}$ if
\begin{equation}
\Phi\left(\mathbf{x}^{(k)} \right) \le \Phi\left(\mathbf{x}^{(k)}+\lambda\mathbf{p}\right)\quad ^{\forall}\lambda \in \mathbb{R}.
\end{equation}
- Note that the optimal $\mathbf{x}^{(k)}$ is the local minimizer of $\Phi$ along $\mathbf{p}$ requiring $\lambda=0$: i.e., no need to move because we are already at the local minimizer.
- This realization leads to the following statement:
\begin{equation}
\frac{\partial\Phi}{\partial \lambda} = \mathbf{p}^{T}\left( \mathbf{A}\mathbf{x}^{(k)} - \mathbf{b} \right) + \lambda \mathbf{p}^{T}\mathbf{A}\mathbf{p} = -\mathbf{p}^{T}\mathbf{r}^{(k)} + \lambda \mathbf{p}^{T}\mathbf{A}\mathbf{p}
\end{equation}
    - Since $\partial \Phi/\partial \lambda$ should be 0 when $\lambda = 0$, $\mathbf{p}^{T}\mathbf{r}^{(k)} = 0$ when $\mathbf{x}^{(k)}$ is optimal.

We have the following general relationship between the current and the next residual:
\begin{equation}
\begin{split}
\mathbf{r}^{(k+1)} &= \mathbf{b} - \mathbf{A}\mathbf{X}^{(k+1)} \\
                   &= \mathbf{b} - \mathbf{A}\left( \mathbf{X}^{(k)} + \alpha_{k}\mathbf{p}^{(k)} \right) \\
                   &= \mathbf{r}^{(k)} -  \alpha_{k}\mathbf{A}\mathbf{p}^{(k)}.
\end{split}
\end{equation}

Since $\mathbf{p}^{(k)} = \mathbf{r}^{(k)}$ in GM,
\begin{equation}
\begin{split}
\mathbf{r}^{(k)T}\mathbf{r}^{(k+1)} &= \mathbf{r}^{(k)T} \left( \mathbf{r}^{(k)} -  \alpha_{k}\mathbf{A}\mathbf{r}^{(k)} \right) \\
                                    &= \mathbf{r}^{(k)T}\mathbf{r}^{(k)} - \alpha_{k}\mathbf{r}^{(k)T}\mathbf{A}\mathbf{r}^{(k)} = 0
\end{split}
\end{equation}
The above equation holds always because of the definition of $\alpha_{k}$.
So, in GM, $\mathbf{r}^{(k)} \perp \mathbf{r}^{(k+1)}$ but in general, $\mathbf{r}^{(k)} \not\perp \mathbf{r}^{(k+2)}$

- The next important question is this: **Are there descent directions that maintain the optimality of iterates?**
    - The answer is **YES**. 

- To fomulate the way of finding them, let's go back to the iteration scheme: $\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \mathbf{q}$.
    - Let's also assume that $\mathbf{x}^{(k)}$ is optimal with respect to a direction $\mathbf{p}$ (i.e., $\mathbf{r}^{(k)}\perp \mathbf{p}$).
    - We are going to require $\mathbf{x}^{(k+1)}$ is still optimal w.r.t. $\mathbf{p}$: i.e., $\mathbf{r}^{(k+1)}\perp \mathbf{p}$
\begin{equation}
\begin{split}
0 &= \mathbf{p}^{T}\mathbf{r}^{(k+1)} = \mathbf{p}^{T} \left( \mathbf{b} - \mathbf{A}\mathbf{x}^{(k+1)} \right) \\
  &= \mathbf{p}^{T} \left( \mathbf{b} - \mathbf{A}\left(\mathbf{x}^{(k)}+\mathbf{q}\right) \right)
  &= \mathbf{p}^{T}\mathbf{r}^{(k)} - \mathbf{p}^{T}\mathbf{A}\mathbf{q}.\\
  &\therefore \mathbf{p}^{T}\mathbf{A}\mathbf{q} = 0.
\end{split}
\end{equation}
- The conclusion is that directions $\mathbf{p}$ and $\mathbf{q}$ should be **A-conjugate** in order to maintain the optimality of $\mathbf{x}^{(k+1)}$ w.r.t. $\mathbf{p}$.

To find a sequence of pairwise A-conjugate directions, we set
\begin{equation}
\mathbf{p}^{(k+1)} = \mathbf{r}^{(k+1)} - \beta_{k}\mathbf{p}^{(k)}\quad k=0,1,\cdots.
\end{equation}

The conjugacy condition gives
\begin{equation}
\mathbf{p}^{(k)T}\mathbf{A}\mathbf{p}^{(k+1)} = \mathbf{p}^{(k)T}\mathbf{A}\left(\mathbf{r}^{(k+1)} - \beta_{k}\mathbf{p}^{(k)} \right) = 0.
\end{equation}

\begin{equation}
\therefore \beta_{k} = \frac{\mathbf{p}^{(k)T}\mathbf{A}\mathbf{r}^{(k+1)}}{\mathbf{p}^{(k)T}\mathbf{A}\mathbf{p}^{(k)}}.
\end{equation}

Note that $\mathbf{p}^{(j)T}\mathbf{A}\mathbf{p}^{(k+1)}=0$ for $j=0,1,\cdots,k$ as we require the descent directions to be.


In summary, for $k=0, 1, \cdots$,
\begin{align}
\mathbf{r}^{(0)} &= \mathbf{b} - \mathbf{A}\mathbf{x}^{(0)},\ \mathbf{p}^{(0)}=\mathbf{r}^{(0)} \\
\alpha_{k} &= \frac{\mathbf{p}^{(k)T}\mathbf{r}^{(k)}}{\mathbf{p}^{(k)T}\mathbf{A}\mathbf{p}^{(k)}} \\
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \alpha_{k} \mathbf{p}^{(k)} \\
\mathbf{r}^{(k+1)} &= \mathbf{r}^{(k)} - \alpha_{k} \mathbf{A} \mathbf{p}^{(k)} \\
\beta_{k} &= \frac{\mathbf{p}^{(k)T}\mathbf{A}\mathbf{r}^{(k+1)}}{\mathbf{p}^{(k)T}\mathbf{A}\mathbf{p}^{(k)}} \\
\mathbf{p}^{(k+1)} &= \mathbf{r}^{(k+1)} - \beta_{k}\mathbf{p}^{(k)}.
\end{align}

- Unlike the classical iterative methods, CG converges to the exact solution within $n$ steps if $\mathbf{A}$ is $n\times n$!
    - Not so useful for large $n$ though.
- Preconditioning is very important for large and complicated problems.
- Multi-grid method: See Ismael-Zadeh and Tackley
    - Superb complexity that is almost linear.

### Brief introduction to Krylov subspace methods

Let's recall the interation scheme:
\begin{equation}
\begin{split}
\mathbf{x}^{(k+1)} &= \mathbf{x}^{(k)} + \alpha_{k}\mathbf{P}^{-1}\mathbf{r}^{(k)} \\
                   &= \mathbf{x}^{(k)} + \alpha_{k}\mathbf{P}^{-1}\left(\mathbf{b}-\mathbf{A}\mathbf{x}^{(k)} \right) \\
                   &= \underbrace{\left( \mathbf{I} - \alpha_{k}\mathbf{P}^{-1}\mathbf{A}\right)}_{\text{Iteration matrix}}\mathbf{x}^{(k)} + \alpha_{k}\mathbf{P}^{-1}\mathbf{b}.
\end{split}
\end{equation}

We can show that if $\mathbf{P}=\mathbf{I}$, 
\begin{equation}
 \mathbf{r}^{(k)} = \left( \prod_{j=0}^{k-1}\left( \mathbf{I} - \alpha_{j}\mathbf{A} \right) \right) \mathbf{r}^{(0)}.
\end{equation}

\begin{equation}
\begin{split}
  \mathbf{r}^{(1)} &= \mathbf{b} - \mathbf{A}\mathbf{x}^{(1)} \\
                   &= \mathbf{b} - \mathbf{A}\left( \left( \mathbf{I} - \alpha_{0}\mathbf{A} \right) \mathbf{x}^{(0)} + \alpha_{0}\mathbf{b} \right) \\
                   &= \left( \mathbf{b} - \mathbf{A}\mathbf{x}^{(0)} \right) - \alpha_{0}\mathbf{A}\left( \mathbf{b} - \mathbf{A}\mathbf{x}^{(0)} \right) \\
                   &= \left( \mathbf{I} - \alpha_{0}\mathbf{A} \right) \mathbf{r}^{(0)}.
\end{split}  
\end{equation}

\begin{equation}
\begin{split}
  \mathbf{r}^{(2)} &= \mathbf{b} - \mathbf{A}\mathbf{x}^{(2)} \\
                   &= \cdots \\
                   &= \left( \mathbf{I} - \alpha_{0}\mathbf{A} \right)\left( \mathbf{I} - \alpha_{1}\mathbf{A} \right) \mathbf{r}^{(0)}.
\end{split}  
\end{equation}

\begin{equation}
\begin{split}
  &\vdots \\
  &\mathbf{r}^{(k)} = p_{k}(\mathbf{A}) \mathbf{r}^{(0)}.
\end{split}  
\end{equation}

Define **Krylov subspace of order $m$** as 
\begin{equation}
\mathcal{K}_{m}(\mathbf{A};\mathbf{v}) = \text{span}\left\{\mathbf{v}, \mathbf{Av}, \mathbf{A}^{2}\mathbf{v}, \cdots,  \mathbf{A}^{m-1}\mathbf{v} \right\}
\end{equation}

Then, $\mathbf{r}^{(k)} \in \mathcal{K}_{k+1}(\mathbf{A};\mathbf{r}^{(0)})$.

Again if $\mathbf{P}=\mathbf{I}$, $\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_{k}\mathbf{r}^{(k)}$. So, we get
\begin{equation}
\mathbf{x}^{(k)} = \mathbf{x}^{(0)} + \sum_{j=0}^{k-1}\alpha_{j}\mathbf{r}^{(j)}.
\end{equation}

Let's define yet another space
\begin{equation}
  W_{k} = \left\{\mathbf{v} \vert \mathbf{v} = \mathbf{x}^{(0)} + \mathbf{y}, \mathbf{y} \in \mathcal{K}_{k}(\mathbf{A}; \mathbf{r}^{(0)}) \right\}.
\end{equation}

A Krylov subspace method looks for an approximate solution of the form
\begin{equation}
  \mathbf{x}^{(k)} = \mathbf{x}^{(0)} + q_{k-1}(\mathbf{A})\mathbf{r}^{(0)} \in W_{k}.
\end{equation}

Let's try to get a little more concrete understanding of this class of methods by realizing that CGM can be considered as a Kyrlov subspace method.

Recall the CGM algorithm: For $k=0, 1, \cdots$,
\begin{equation}
\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_{k} \mathbf{p}^{(k)}.
\end{equation}

So, 
\begin{align}
\mathbf{x}^{(1)} &= \mathbf{x}^{(0)} + \alpha_{0} \mathbf{p}^{(0)} \\
\mathbf{x}^{(2)} &= \mathbf{x}^{(1)} + \alpha_{0} \mathbf{p}^{(1)} \\
                 &= \mathbf{x}^{(0)} + \alpha_{0} \mathbf{p}^{(0)} + \alpha_{0} \mathbf{p}^{(1)} \\
                 &\vdots \\
\mathbf{x}^{(k)} &= \mathbf{x}^{(0)} + \sum_{j=0}^{k-1}\alpha_{j}\mathbf{p}^{(j)}.
\end{align}

To see the equivalence, we have to determine that
\begin{equation}
 \sum_{j=0}^{k-1}\alpha_{j}\mathbf{p}^{(j)} = q_{k-1}(\mathbf{A})\mathbf{r}^{(0)}.
\end{equation}
In other words, is the following true?
\begin{equation}
 \text{span}\left\{ \mathbf{p}^{(0)}, \mathbf{p}^{(1)}, \ldots, \mathbf{p}^{(k-1)} \right\} = \mathcal{K}_{k}(\mathbf{A};\mathbf{r}^{(0)}).
\end{equation}
The answer is YES.

According to the CGM algorithm,
\begin{align}
\mathbf{x}^{(1)} &= \mathbf{x}^{(0)} + \alpha_{0}\mathbf{p}^{(0)} \\
\mathbf{r}^{(1)} &= \mathbf{r}^{(0)} - \alpha_{0} \mathbf{A} \mathbf{p}^{(0)} \\
\mathbf{p}^{(1)} &= \mathbf{r}^{(1)} - \beta_{0}\mathbf{p}^{(0)} \\
                 &= \mathbf{r}^{(0)} - \alpha_{0} \mathbf{A} \mathbf{p}^{(0)} - \beta_{0}\mathbf{p}^{(0)} \\
                 &= \left[(1-\beta_{0})\mathbf{I}-\alpha_{0}\mathbf{A}\right]\mathbf{r}^{(0)} \\
                 & \\
\mathbf{x}^{(2)} &= \mathbf{x}^{(0)} + \alpha_{0}\mathbf{p}^{(0)} + \alpha_{1}\mathbf{p}^{(1)} \\
\mathbf{r}^{(2)} &= \mathbf{r}^{(1)} - \alpha_{1} \mathbf{A} \mathbf{p}^{(1)} \\
                 &= \left( \mathbf{r}^{(0)} - \alpha_{0} \mathbf{A} \mathbf{p}^{(0)} \right) - \alpha_{1} \mathbf{A} \left[(1-\beta_{0})\mathbf{I}-\alpha_{0}\mathbf{A}\right]\mathbf{r}^{(0)} \\
\mathbf{p}^{(2)} &= \mathbf{r}^{(2)} - \beta_{1}\mathbf{p}^{(1)} \\
                 &= q_{2}(\mathbf{A})\mathbf{r}^{(0)} \\
                 &\vdots
\end{align}

- The basis for $\mathcal{K}_{m}(\mathbf{A};\mathbf{r}^{(0)})$ are not a well-conditioned one.
- An equivalent orthonormal basis is used instead (e.g., the Arnoldi algorithm).
- The practical question is how to find a suitable linear combination of the bases.
    1. Full Orthogonalization Method (FOM): compute $\mathbf{x}^{(k)} \in W_{k}$ such that
    \begin{equation}
      \mathbf{v}^{T}(\mathbf{b}-\mathbf{A}\mathbf{x}^{(k)}) = 0\ \text{for}\ ^{\forall}\mathbf{v} \in \mathcal{K}_{k}(\mathbf{A};\mathbf{r}^{(0)}).
    \end{equation}
        - CGM is an FOM!
    2. Generalized Minimum Residual Method (GMRES): compute $\mathbf{x}^{(k)} \in W_{k}$ minimizing the Euclidean norm of $\mathbf{r}^{(k)}$:
    \begin{equation}
      \left\lVert \mathbf{b}-\mathbf{A}\mathbf{x}^{(k)} \right\rVert_{2} = \min_{\mathbf{v}\in W_{k}} \left\lVert \mathbf{b}-\mathbf{A}\mathbf{v} \right\rVert_{2}.
    \end{equation}
        - It takes a little more effort to understand how this criterion leads to a programmable iteration algorithm.

- Krylov subspace methods are known for their efficienty and flexibility in solving large sparse system. 
    - Even for non-symmetric $\mathbf{A}$.
- For more, also see "Numerical Linear Algebra with Applications : Using MATLAB" by W. Ford (First edition. London : Academic Press. 2015).
    - The university library has an ebook version: http://ezproxy.memphis.edu/login?url=http://search.ebscohost.com/login.aspx?direct=true&db=e000xna&AN=485990&site=ehost-live&ebv=EB&ppid=pp_146